In [21]:
!nvidia-smi

Mon Apr 14 22:40:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             31W /   70W |    5824MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [22]:
!pip install transformers
!pip install langchain
!pip install chromadb
!pip install pypdf
!pip install xformers
!pip install InstructorEmbedding
!pip install pdf2image

In [23]:
!pip install torch

In [24]:
import torch
print(torch.__version__)


2.6.0+cu124


In [25]:
!pip install auto_gptq

In [26]:
!pip install langchain_community

import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [27]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [28]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


In [29]:
!gdown 1mmA_ReCmsJiujVWLOkcxKRDQhp9z6DVE -O pdfs/Film-Contracts.pdf
!gdown 1ckb5NUrg3d3O35-U1kluS59LGMhMqVkt -O pdfs/The-Business-of-Film_3rd.pdf


Downloading...
From: https://drive.google.com/uc?id=1mmA_ReCmsJiujVWLOkcxKRDQhp9z6DVE
To: /content/pdfs/Film-Contracts.pdf
100% 33.6M/33.6M [00:00<00:00, 42.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ckb5NUrg3d3O35-U1kluS59LGMhMqVkt
To: /content/pdfs/The-Business-of-Film_3rd.pdf
100% 5.35M/5.35M [00:00<00:00, 37.3MB/s]


In [30]:
loader = PyPDFDirectoryLoader("pdfs")
docs = loader.load()
len(docs)

1043

In [31]:
for doc in docs:
  print(doc.metadata)

{'producer': 'Adobe Acrobat Pro DC (32-bit) 22.1.20117', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2022-09-09T19:14:57+05:30', 'author': 'Stephen R. Greenwald', 'keywords': 'business of media; OTT; crowdfunding; film distribution; media business; media distribution; film finance; media entrepreneurship; AR; VR', 'moddate': '2022-09-14T14:37:46+05:30', 'subject': 'This updated third edition introduces readers to the business of film at every stage of the filmmaking lifecycle A practical, hands-on guide exploring streaming, development, financing trends, regional/global/online distribution, shifting business models, exhibition, multi-platform delivery, marketing, VR/AR, accounting, and more.', 'title': 'The Business of Film; A Practical Introduction', 'source': 'pdfs/The-Business-of-Film_3rd.pdf', 'total_pages': 398, 'page': 0, 'page_label': 'Cover'}
{'producer': 'Adobe Acrobat Pro DC (32-bit) 22.1.20117', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '20

In [32]:
#from huggingface_hub import snapshot_download
#snapshot_download(repo_id="hkunlp/instructor-large")

!pip install langchain-huggingface

from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'device': DEVICE}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs=model_kwargs)


In [33]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    chunk_size=1024,
    chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

3380

In [34]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 3min 24s, sys: 1.11 s, total: 3min 25s
Wall time: 3min 38s


## Llama 2 13B

In [36]:
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM # Import AutoGPTQForCausalLM from auto_gptq


model_name_or_path = "MaziyarPanahi/Mistral-7B-Instruct-v0.1-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="main",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads a

quantize_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.


In [37]:
!nvidia-smi

Mon Apr 14 22:51:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             33W /   70W |    5406MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [38]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.

If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [39]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [40]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

Device set to use cuda:0
The model 'MistralGPTQForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTN

In [41]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

<ipython-input-41-c1cd673e8ff4>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


In [42]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [43]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [44]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
chain_type="stuff",
    #retriever=db.as_retriever(search_kwargs={"k": 2}),
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Chat

In [45]:
result = qa_chain( 'The process of transforming a film from a script to a commercial moviegoes through five steps:'  )

<ipython-input-45-f3fbe6f9cfe6>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain( 'The process of transforming a film from a script to a commercial moviegoes through five steps:'  )
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I apologize, but it seems like there is some missing information in the provided text. Could you please provide more context or clarify the question?


In [46]:
print(result["source_documents"])

[Document(metadata={'title': 'The Business of Film; A Practical Introduction', 'author': 'Stephen R. Greenwald', 'subject': 'This updated third edition introduces readers to the business of film at every stage of the filmmaking lifecycle A practical, hands-on guide exploring streaming, development, financing trends, regional/global/online distribution, shifting business models, exhibition, multi-platform delivery, marketing, VR/AR, accounting, and more.', 'keywords': 'business of media; OTT; crowdfunding; film distribution; media business; media distribution; film finance; media entrepreneurship; AR; VR', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2022-09-09T19:14:57+05:30', 'page': 27, 'source': 'pdfs/The-Business-of-Film_3rd.pdf', 'total_pages': 398, 'moddate': '2022-09-14T14:37:46+05:30', 'producer': 'Adobe Acrobat Pro DC (32-bit) 22.1.20117', 'page_label': '9'}, page_content='newly minted producers were first and foremost businessmen, with a focus \non profit, not

In [ ]:
result = qa_chain( 'list the key characteristics of the film industry'  )

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The key characteristics of the film industry are:

• film as intellectual property
• managing creative talent
• film as a global product
• film takes a long time to develop, produce, and market
• market structure—oligopoly
• cyclical
• price elasticity


In [ ]:
print(result["source_documents"])

In [ ]:
result = qa_chain( 'give me a contract template for submission release')



In [ ]:
print(result["source_documents"][0].page_content)

## References
https://github.com/PanQiWei/AutoGPTQ/
